In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.dpi'] = 125
plt.rcParams['figure.facecolor'] = 'white'

# If package installed use:
# from pypekeris import pekeris, pekeris_broadband

# Otherwise:

import os
import sys 

module_path = os.path.abspath(os.path.join('/Users/gilaverbuch/dev/WavePropagation/pypekeris'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.pypekeris.pypekeris import pekeris, pekeris_broadband


# Problem geometry

Two-layer liquid half space. Layer 1 (0<z<D) has a speed of sound c1. Layer 2 (z>D) has a speed of sound c2. Both source and receiver ar placed within the first layer. c2>c1. 

--------------------------------------------------------------------------------- z=0, pressure release surface

c1


*(0, z_s)




                                                           *(r,z)

--------------------------------------------------------------------------------- z=D, continuitu of P and w

c2



The period equation $\cot (k_0 \mu_1 d)$ = $-|\mu_2|/\mu_1$ -> Discrete spectrum. It implicitly defines the relationship between the priod and phase velocity.

In [ ]:
P = pekeris()
P.calc_parameters()
P.print_parameters()

In [ ]:
P.plot_discrete_modes()

In [ ]:
P.plot_modes_shape()

In [ ]:
P.calc_2D_field()
P.plot_TL()

In [ ]:
P.freq = 500
P.dz = 0.1
P.r = 1000
P.calc_parameters()
P.calc_2D_field()
P.plot_TL()

In [ ]:
P.nq=2e4
P.calc_parameters()
P.plot_dispersion(fmax=250, df=0.1)

In [ ]:
%%time
tr_s, tr_r = pekeris_broadband(fmax=4.9e2, r=2e4, dt=1e-3, num_mode=5)

In [ ]:
tr_r.plot();

In [ ]:
fig = tr_s.spectrogram(wlen=0.5, per_lap=0.95, mult=2, log=False, dbscale=False, show=False);
fig.set_figheight(10)
fig.set_figwidth(15)
plt.ylim(0,300)
plt.xlim(0,15)
plt.show()

In [ ]:
fig = tr_r.spectrogram(wlen=0.5, per_lap=0.95, mult=2, log=False, dbscale=False, show=False);
fig.set_figheight(10)
fig.set_figwidth(15)
plt.ylim(0,500)
plt.xlim(15,25)
plt.show()